### Usando Visualizações como Consultas Dinâmicas

Embora os widgets de interface padrão mostrem os valores _possíveis_ para os parâmetros de consulta, eles não visualizam a _distribuição_ desses valores. Também podemos desejar usar interações mais ricas, como seleções de vários valores ou intervalos, em vez de widgets de entrada que selecionam apenas um único valor de cada vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e oferecer suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e utilizar uma seleção de intervalo para realçar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos de tempo. Vê alguma evidência de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Qual é a relação entre o ano e as classificações dos críticos?)*.

_Os anos vão de 1930 a 2040! Os futuros filmes estão em pré-produção ou  existem erros "fora de um século"? Além disso, dependendo do fuso horário em que estamos, há um pequeno aumento em 1969 ou 1970. Porque isso acontece? (Veja o final do notebook para uma explicação!)_


In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limitar a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmico
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modificar a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

alt.VConcatChart(...)

O exemplo acima fornece consultas dinâmicas utilizando uma _seleção vinculada_ entre gráficos:

- Criamos uma seleção `interval` (`brush`), e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando num intervalo de seleção unidimensional.
- Registramos o `brush` com o nosso histograma de filmes por ano através de `.add_selection(brush)`.
- Usamos `brush` numa codificação condicional para ajustar a opacidade (`opacity`) do scatter plot.

Esta técnica de interação de selecionar elementos num gráfico e ver os destaques ligados num ou mais gráficos é conhecida como seleção &amp; ligação ([_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking)).

## Panning &amp; Zooming

O _scatter plot_ da classificação dos filmes é um pouco confuso em alguns sítios, tornando difícil examinar pontos em regiões mais densas. Utilizando as técnicas de interação de _panning_ e _zooming_, podemos inspecionar regiões densas mais de perto.

Comecemos pensando em alguma forma de expressar o _panning_ e o _zooming_ utilizando as seleções do Altair. O que define a "janela de visualização" de um gráfico? _Domínios de escala dos eixos!_

Podemos alterar os domínios de escala para modificar o intervalo visualizado de valores de dados. Para fazer isso interativamente, podemos vincular uma seleção `interval` aos domínios de escala com o código `bind='scales'`. O resultado é que, em vez de uma seleção de intervalo que podemos arrastar e ampliar, podemos arrastar e ampliar toda a área de plotagem!

_No gráfico abaixo, clique e arraste para deslocar (traduzir) a visualização, ou role para ampliar (escalar) a visualização. O que é que você pode descobrir sobre a precisão dos valores de classificação fornecidos?_


In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

Ao fazer _zooming_, podemos ver que os valores de classificação têm uma precisão limitada! As classificações do Rotten Tomatoes são números inteiros, enquanto as classificações do IMDB são truncadas em décimos. Como resultado, existe uma sobreposição de gráficos mesmo quando fazemos zoom, com vários filmes compartilhando os mesmos valores de classificação.

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço seja reservada para os ticks e rótulos do eixo. Por que fazer isso? Quando fazemos pan e zoom, os rótulos dos eixos podem mudar e fazer com que a posição do título do eixo se desloque. Ao definir uma extensão mínima, podemos reduzir os movimentos de distração no gráfico. Tente alterar o valor de `minExtent`, por exemplo, definindo-o como zero, e depois diminua o zoom para ver o que acontece quando rótulos de eixo mais longos entram na visualização.

O Altair também inclui uma forma abreviada de adicionar _panning_ e _zooming_ a um gráfico. Ao invés de criar diretamente uma seleção, você pode chamar `.interactive()` para que o Altair gere automaticamente uma seleção de intervalos vinculada às escalas do gráfico:


In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrāo, as ligações de escala para seleções incluem os canais de codificação `x` e `y`. E se quisermos limitar o _panning_ e o _zoomimg_ ao longo de uma única dimensão? Podemos chamar `encodings=['x']` para restringir a seleção apenas ao canal `x`:


In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

_Quando se faz zooming apenas ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente a nossa perceção das relações nos dados. [Escolher uma proporção de aspecto adequada](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante do design de visualização!_


## Navegação: Visão Geral + Detalhe

Ao usar _panning_ e _zooming_, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada de _visão geral + detalhe_ utiliza, em vez disso, uma visão geral de exibição para mostrar _todos_ os dados, ao mesmo tempo que suporta seleções pan e zoom em uma exibição de foco separado.

Abaixo temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, os dois gráficos mostram o mesmo intervalo de dados. Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos.


In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

Ao contrário do nosso caso anterior, panning &amp; zooming, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Em vez disso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para isso, atualizamos o canal de codificação `x` do nosso gráfico de foco, definindo a propriedade domínio (`domain`) da escala para fazer referência à nossa seleção `brush`. Se nenhum intervalo for definido (a seleção está vazia), o Altair ignora o _brush_ e usa os dados subjacentes para determinar o domínio. Quando um intervalo de _brush_ é criado, o Altair o utiliza como o domínio da escala para o gráfico de foco.